In [13]:

import urllib.request
urllib.request.urlretrieve('http://www.atarimania.com/roms/Roms.rar','Roms.rar')
!pip install unrar
!unrar x Roms.rar
!mkdir rars
!mv HC\ ROMS.zip   rars
!mv ROMS.zip  rars
!python -m atari_py.import_roms rars
!pip install tensorflow==2.4
!pip install keras==2.2.4
!pip install Pillow
!pip install h5py
!pip install keras-rl2
!pip install gym
!pip install gym[atari]

#!/usr/bin/env python3
""" Trains an agent to play Breakout """
import numpy as np
import gym

import tensorflow.keras as K
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense, Permute
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import EpsGreedyQPolicy, LinearAnnealedPolicy
from rl.callbacks import FileLogger
from rl.core import Processor
from PIL import Image

import tensorflow.python.util.deprecation as deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False


INPUT_SHAPE = (84, 84)


class AtariProcessor(Processor):
    """ Preprocess images as per deepmind paper """
    def process_observation(self, observation):
        """ Process observation """
        assert observation.ndim == 3
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):
        """ Convert batch to float32 """
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        """ Process reward """
        return np.clip(reward, -1., 1.)

env = gym.make('Breakout-v0')
nb_actions = env.action_space.n

inputs = Input(shape=((4,) + INPUT_SHAPE))
x = Permute((2, 3, 1))(inputs)
x = Conv2D(32, 8, strides=4, activation='relu')(x)
x = Conv2D(64, 4, strides=2, activation='relu')(x)
x = Conv2D(64, 2, strides=3, activation='relu')(x)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
action = Dense(nb_actions, activation='linear')(x)
model = K.Model(inputs=inputs, outputs=action)
model.summary()

memory = SequentialMemory(limit=1000000, window_length=4)
policy = LinearAnnealedPolicy(
    EpsGreedyQPolicy(),
    attr='eps',
    value_max=1.,
    value_min=.1,
    value_test=0.05,
    nb_steps=1000000
)
dqn = DQNAgent(
    model=model,
    nb_actions=nb_actions,
    memory=memory,
    nb_steps_warmup=50000,
    target_model_update=10000,
    policy=policy,
    processor=AtariProcessor(),
    gamma=.99,
    train_interval=4,
    delta_clip=1.
)
dqn.compile(K.optimizers.Adam(lr=.00025), metrics=['mae'])

dqn.fit(
    env,
    nb_steps=1750000,
    visualize=False,
    verbose=2
)

dqn.save_weights('policy.h5', overwrite=True)



UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from Roms.rar

Extracting  HC ROMS.zip                                                   36%  OK 
Extracting  ROMS.zip                                                      74% 99%  OK 
All OK
mkdir: cannot create directory ‘rars’: File exists
copying adventure.bin from HC ROMS/BY ALPHABET (PAL)/A-G/Adventure (PAL).bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/adventure.bin
copying air_raid.bin from HC ROMS/BY ALPHABET (PAL)/A-G/Air Raid (PAL).bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/air_raid.bin
copying alien.bin from HC ROMS/BY ALPHABET (PAL)/A-G/REMAINING NTSC ORIGINALS/Alien.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/alien.bin
copying crazy_climber.bin from HC ROMS/BY ALPHABET (PAL)/A-G/REMAINING NTSC ORIGINALS/Crazy Climber.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/crazy_climber.bin
copying 

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


     167/1750000: episode: 1, duration: 0.871s, episode steps: 167, steps per second: 192, episode reward:  0.000, mean reward:  0.000 [ 0.000,  0.000], mean action: 1.611 [0.000, 3.000],  loss: --, mae: --, mean_q: --, mean_eps: --
     473/1750000: episode: 2, duration: 0.973s, episode steps: 306, steps per second: 314, episode reward:  2.000, mean reward:  0.007 [ 0.000,  1.000], mean action: 1.510 [0.000, 3.000],  loss: --, mae: --, mean_q: --, mean_eps: --
     950/1750000: episode: 3, duration: 1.514s, episode steps: 477, steps per second: 315, episode reward:  5.000, mean reward:  0.010 [ 0.000,  1.000], mean action: 1.568 [0.000, 3.000],  loss: --, mae: --, mean_q: --, mean_eps: --
    1123/1750000: episode: 4, duration: 0.566s, episode steps: 173, steps per second: 306, episode reward:  0.000, mean reward:  0.000 [ 0.000,  0.000], mean action: 1.671 [0.000, 3.000],  loss: --, mae: --, mean_q: --, mean_eps: --
    1303/1750000: episode: 5, duration: 0.574s, episode steps: 180, 

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


   50096/1750000: episode: 200, duration: 3.522s, episode steps: 226, steps per second:  64, episode reward:  1.000, mean reward:  0.004 [ 0.000,  1.000], mean action: 1.535 [0.000, 3.000],  loss: 0.001523, mae: 0.014972, mean_q: 0.024097, mean_eps: 0.954957
   50269/1750000: episode: 201, duration: 3.228s, episode steps: 173, steps per second:  54, episode reward:  0.000, mean reward:  0.000 [ 0.000,  0.000], mean action: 1.503 [0.000, 3.000],  loss: 0.002493, mae: 0.012258, mean_q: 0.023317, mean_eps: 0.954836
   50438/1750000: episode: 202, duration: 3.103s, episode steps: 169, steps per second:  54, episode reward:  0.000, mean reward:  0.000 [ 0.000,  0.000], mean action: 1.763 [0.000, 3.000],  loss: 0.003041, mae: 0.015330, mean_q: 0.028478, mean_eps: 0.954681
   50728/1750000: episode: 203, duration: 5.356s, episode steps: 290, steps per second:  54, episode reward:  2.000, mean reward:  0.007 [ 0.000,  1.000], mean action: 1.528 [0.000, 3.000],  loss: 0.003706, mae: 0.014555, m